# SDG Target Reranker
This notebook takes the top-predicted SDG goal from your LLM pipeline and reranks its detailed SDG targets using zero-shot classification.

In [5]:
import pickle
from transformers import pipeline
from typing import List, Tuple
import torch

###  Load SDG Targets

In [6]:
def load_sdg_targets(target_path: str) -> dict:
    with open(target_path, 'rb') as f:
        targets_df = pickle.load(f)
    grouped = targets_df.groupby(targets_df.columns[0])[targets_df.columns[1]].apply(list).to_dict()
    return grouped  # {Goal 6: [target1, target2, ...]}

###  Rerank Targets within a Goal

In [7]:
def rerank_targets_within_goal(text: str, targets: List[str], classifier, top_k: int = 3) -> List[Tuple[str, float]]:
    result = classifier(text, candidate_labels=targets, multi_label=True)
    paired = list(zip(result['labels'], result['scores']))
    return sorted(paired, key=lambda x: -x[1])[:top_k]

### Main Function to Explain Top Goal with Targets

In [8]:
def explain_top_goal_with_targets(text: str, top_goal: str, target_path: str, top_k: int = 3):
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0 if torch.cuda.is_available() else -1)
    targets_by_goal = load_sdg_targets(target_path)
    targets = targets_by_goal.get(top_goal, [])
    if not targets:
        return [("No targets found for this goal", 0.0)]
    return rerank_targets_within_goal(text, targets, classifier, top_k)

###  Example Usage

In [9]:
import json

with open("results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# Example: use the first patent
text = results[0]['text']
top_goal = results[0]['predictions'][0][0]  # highest scoring SDG

print(" Using:")
print("Text:", text)
print("Top Goal:", top_goal)


 Using:
Text: A system for purifying water using solar energy.
Top Goal: Clean Water and Sanitation


In [10]:
reranked_targets = explain_top_goal_with_targets(text, top_goal, "sgd_targets.dat")

for t, s in reranked_targets:
    print(f" - {t} ({s:.2f})")


Device set to use cpu


 - No targets found for this goal (0.00)
